In [42]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import datetime
import re
import string
import nltk
from nltk.corpus import stopwords
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV,StratifiedKFold,RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import f1_score
from wordcloud import WordCloud,STOPWORDS
from sklearn.model_selection import train_test_split
from nltk.stem.snowball import SnowballStemmer
import nltk, os, re, string, collections
import matplotlib
from  imageio import imread
from nltk.util import ngrams
import itertools
import spacy
from sklearn.preprocessing import MultiLabelBinarizer, Binarizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from collections import Counter
from sklearn.model_selection import train_test_split,KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVR
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics
import lightgbm as lgb
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import cross_val_predict, cross_val_score
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)
import gc
from mlxtend.classifier import StackingClassifier
from bayes_opt import BayesianOptimization #pip install bayesian_optimization
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, AdaBoostClassifier, VotingClassifier

In [45]:
train = pd.read_csv('original_data/train.csv')
test  = pd.read_csv('original_data/test.csv')

# Limpieza texto

In [46]:
def getStopwordsList():
    fileNamesList = ["texts/99webtools.txt", "texts/atire_ncbi.txt", "texts/atire_puurula.txt", "texts/azure.txt", 
                 "texts/bbalet.txt", 
                 "texts/bow_short.txt", "texts/choi_2000naacl.txt", "texts/cook1988_function_words.txt", 
                 "texts/corenlp_acronym.txt", 
                 "texts/corenlp_hardcoded.txt", "texts/corenlp_stopwords.txt", "texts/datasciencedojo.txt", 
                 "texts/deeplearning4j.txt", 
                 "texts/dkpro.txt", "texts/mongodb.txt", "texts/galago_inquery.txt", "texts/gate_keyphrase.txt", 
                 "texts/gensim.txt", 
                 "texts/glasgow_stop_words.txt", "texts/indri.txt", "texts/kevinbouge.txt", "texts/lexisnexis.txt",
                 "texts/lingpipe.txt", 
                 "texts/mallet.txt", "texts/mysql_innodb.txt", "texts/mysql_myisam.txt", "texts/galago_rmstop.txt", 
                 "texts/atire_ncbi.txt", 
                 "texts/galago_rmstop.txt", "texts/nltk.txt", "texts/okapiframework.txt", "texts/okapi_cacm_expanded.txt", 
                 "texts/onix.txt", 
                 "texts/ovid.txt", "texts/postgresql.txt", "texts/pubmed.txt", "texts/quanteda.txt", "texts/r_tm.txt", 
                 "texts/ranksnl_large.txt", 
                 "texts/reuters_wos.txt", "texts/rouge_155.txt", "texts/scikitlearn.txt", "texts/smart.txt", 
                 "texts/snowball_expanded.txt", 
                 "texts/spacy.txt", "texts/spark_mllib.txt", "texts/sphinx_mirasvit.txt", "texts/t101_minimal.txt", 
                 "texts/taporware.txt", 
                 "texts/terrier.txt", "texts/tonybsk_1.txt", "texts/tonybsk_6.txt", "texts/voyant_taporware.txt", 
                 "texts/weka.txt", 
                 "texts/xapian.txt", "texts/xpo6.txt", "texts/zettair.txt"]
    stopwordsList = []
    for fileName in fileNamesList:
        file = open(fileName, "r", encoding="utf8")
        for line in file:
            stripped_line = line. strip()
            line_list = stripped_line
            if line_list not in stopwordsList:
                stopwordsList.append(line_list)
        file.close()
    return stopwordsList

stopwordsList = getStopwordsList()

print(len(stopwordsList))

2253


In [47]:
def clean_text_strict(text):
    text = text.lower()
    text = text.replace('\d+', '')
    invalid_chars = ['#','|','@','!','?','-','_','[',']','%','&',':','.',',',"''",'/','https','(','//t',')','http',
                 ';','\'']
    for char in invalid_chars:
        if char in text:
            text = text.replace(char,' ')
    #removes url and tags
    text = re.sub(r"(?:\@|https?\://)\S+", " ",  text)
    return text

In [48]:
# FUNCIONES AUXILIARES

w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

def prettify(word):
    word = re.sub(r'[^A-Za-z]', " ",  word)
    word = re.sub(r'\b\w\b',' ', word)
    word = re.sub(r'\s+', ' ', word)
    return word

def cleanStopwords(tweet):
    tweet = tweet.split(' ')
    cleanTweet=""
    for word in tweet:
        if word not in stopwordsList:
            cleanTweet = cleanTweet + " " + word
    return cleanTweet

def clean_text_non_strict(text): 
    tw = " "
    words = text.split(' ')
    for word in words:
        word = ''.join(filter(lambda x: x in set(string.printable), word))
        word = word.replace("\n"," ")
        word = word.replace('û',"")
        word = word.replace('Û',"")
        #word = word.replace('_','')
        #word = word.replace("\"",'')
        #word = word.strip('.')
        #word = word.strip(',')
        #word = word.strip(':')
        tw += word + " "
    return tw

In [ ]:
nlp = spacy.load('en', disable=['parser', 'ner'])
def lemmatize_text_v2(text):
    if text == '' or pd.isnull(text):
        return text
    else:
        doc = nlp(text)
        return " ".join([token.lemma_ for token in doc])

In [ ]:
#FUNCIONES AUXILIARES: Limpieza de tweet

# Función que saca los links

def quitar_link_twitter(tweet):
    res = []
    for w in tweet.split(" "):
        if (("http" not in w) and ("https" not in w)):
            res.append(w)
    return (" ").join(res)

# Función que saca las menciones

def quitar_mencion_twitter(tweet):
    res = []
    for w in tweet.split(" "):
        if("@" not in w):
            res.append(w)
    return (" ").join(res)

def quitar_simbolo(tweet, simbolo):
    text = ""
    for c in tweet:
        if(c!=simbolo):
            text+=c
    return text

def agregar_espacio_ente_comas(tweet):
    
    tweer_sin_coma = (" ").join(tweet.split(","))

    return (" ").join(tweer_sin_coma.split("."))

def capitalize_each_word(string):
    
    text = ""
    for w in string.split(" "):
        if(len(w)>3):
            text += w.capitalize()+" "
    return text

def limpiar_tweet_location(tweet):
    res = tweet
    func = [quitar_mencion_twitter, quitar_link_twitter, agregar_espacio_ente_comas, lambda x: quitar_simbolo(x, "#"), capitalize_each_word]
    for f in func:
        res = f(res)
    return res

def to_string(a_list, delimeter):
    res = ""
    
    for e in sorted(filter(None, a_list)):
        res += str(e)+delimeter
    
    if(len(res)==0): return None
    return res[: -len(delimeter)]

In [ ]:
def addKeywordToText(x):
    y=pd.isnull(x.keyword)
    if (y == True):
        return x['text']
    else:
        return (x['keyword'] + ' ' + x['text'])

In [ ]:
def addLocationToText(x):
    if (x.location == ' '):
        return x['text_w_location']
    else:
        return (x['location'] + ' ' + x['text_w_location'])

### Text + Keyword [baja un toque el score]

In [ ]:
'''train["keyword"] = train["keyword"].str.replace('%20',' ')
train['text_w_keyword'] = train.apply(lambda x: addKeywordToText(x), axis=1)

test["keyword"] = test["keyword"].str.replace('%20',' ')
test['text_w_keyword'] = test.apply(lambda x: addKeywordToText(x), axis=1)

train.text_w_keyword = train["text_w_keyword"].apply(lambda x: clean_text_strict(x))
test.text_w_keyword  = test["text_w_keyword"].apply(lambda x: clean_text_strict(x))

train.text_w_keyword = train.text_w_keyword.apply(prettify)
test.text_w_keyword  = test.text_w_keyword.apply(prettify)

train.text_w_keyword = train.text_w_keyword.apply(lemmatize_text_v2)
test.text_w_keyword  = test.text_w_keyword.apply(lemmatize_text_v2)

train.text_w_keyword = train.text_w_keyword.apply(cleanStopwords)
test.text_w_keyword  = test.text_w_keyword.apply(cleanStopwords)'''

### Text + Location [baja score :(]

In [ ]:
'''train.location = train.location.fillna('')
train.location = train.location.apply(lambda x:limpiar_tweet_location(x))
train['text_w_location'] = train.apply(lambda x: addLocationToText(x), axis=1)

test.location = test.location.fillna('')
test.location = test.location.apply(lambda x:limpiar_tweet_location(x))
test['text_w_location'] = test.apply(lambda x: addLocationToText(x), axis=1)

train.text_w_location = train["text_w_location"].apply(lambda x: clean_text_strict(x))
test.text_w_location  = test["text_w_location"].apply(lambda x: clean_text_strict(x))

train.text_w_location = train.text_w_location.apply(prettify)
test.text_w_location  = test.text_w_location.apply(prettify)

train.text_w_location = train.text_w_location.apply(lemmatize_text_v2)
test.text_w_location  = test.text_w_location.apply(lemmatize_text_v2)

train.text_w_location = train.text_w_location.apply(cleanStopwords)
test.text_w_location  = test.text_w_location.apply(cleanStopwords)'''

In [ ]:
train.text = train["text"].apply(lambda x: quitar_link_twitter(x)) #Por algun motivo esto baja un poquito el score
test.text  = test["text"].apply(lambda x: quitar_link_twitter(x))

train.text = train["text"].apply(lambda x: quitar_mencion_twitter(x)) #Por algun motivo esto baja un poquito el score
test.text  = test["text"].apply(lambda x: quitar_mencion_twitter(x))

train.text = train["text"].apply(lambda x: clean_text_non_strict(x))
test.text  = test["text"].apply(lambda x: clean_text_non_strict(x))

train.text = train["text"].apply(lambda x: clean_text_strict(x))
test.text  = test["text"].apply(lambda x: clean_text_strict(x))

In [ ]:
train.text = train.text.apply(prettify)
test.text  = test.text.apply(prettify)

In [49]:
# a veces baja score
#train.text = train.text.apply(lemmatize_text_v2)
#test.text  = test.text.apply(lemmatize_text_v2)

In [75]:
train.text

0        -PRON- deed reason earthquake allah forgive -...
1                           forest fire ronge sask canada
2        resident shelter notify officer evacuation sh...
3           people receive wildfire evacuation california
4                   photo ruby alaska smoke wildfire pour
                              ...                        
7608     giant crane hold bridge collapse nearby stfmb...
7609       aria ahrary thetawniest control wild fire c...
7610                        utc volcano hawaii zdtoyd ebj
7611     police bike collide portugal bike rider suffe...
7612       raze northern california wildfire abc ymy rskq
Name: text, Length: 7613, dtype: object

In [51]:
train.text = train.text.apply(cleanStopwords)
test.text  = test.text.apply(cleanStopwords)

In [52]:
train.text

0       our deeds are the reason of this earthquake ma...
1                   forest fire near la ronge sask canada
2       all residents asked to shelter in place are be...
3        people receive wildfires evacuation orders in...
4       just got sent this photo from ruby alaska as s...
                              ...                        
7608    two giant cranes holding bridge collapse into ...
7609     aria ahrary thetawniest the out of control wi...
7610               utc km of volcano hawaii co zdtoyd ebj
7611    police investigating after an bike collided wi...
7612    the latest more homes razed by northern califo...
Name: text, Length: 7613, dtype: object

# Modelos

In [58]:
def save_prediction_accuracy(modelo,x_train,y_train,x_test,y_test):
    clf = modelo
    clf.fit(x_train, y_train)
    predictions = clf.predict(x_test)
    with open("prediction_history.csv", "a") as myfile:
        #round(clf.score(X_train, y_train)*100)
        train_prediction = str((clf.score(X_train, y_train)*100))
        print('Score para x_train: '+ train_prediction)
        test_prediction = str((accuracy_score(y_test,predictions)*100))
        print('Score para x_test: '+ test_prediction)
        params = str(modelo)
        print('Hiperparametros: '+ str(modelo))
        myfile.write(params+','+test_prediction+","+train_prediction+","+str(datetime.datetime.now())+"\n")

In [59]:
def save_prediction(model, test, name):
    predicted = model.predict(test)
    sample_submission.target = predicted
    sample_submission.to_csv('predictions/'+name+'.csv', index=None)
    return

In [60]:
sample_submission = pd.read_csv('original_data/sample_submission.csv')

In [61]:
y = train.target

In [62]:
#X_train, X_test, y_train, y_test = train_test_split(train_vec,y,test_size=0.2,random_state=2020)
X_train, X_test, y_train, y_test = train_test_split(train.text, y, test_size=0.2, random_state=2020)

## Estimación sobre algunos modelos básicos

In [63]:
def runClassifiers(vectorizer, X_train, y_train, X_test):
    classifiers =[LogisticRegression(), MultinomialNB(alpha=1),DecisionTreeClassifier(),
        KNeighborsClassifier(n_neighbors=7, weights = 'distance',p = 1),RandomForestClassifier(), 
                  SGDClassifier(loss='hinge', penalty='l2', alpha=0.001, random_state=42, max_iter=5, tol=None),
                 MLPClassifier()]
    classifier_names = ['Logistic Regression','MultinomialNB','Decision-Tree Classifier','K-Neighbors Classifier',
                       'Random-Forest Classifier','SGDC Classifier', 'MLP Classifier']
    i=-1
    for classifier in classifiers:
        i = i + 1
        text_clf = Pipeline([
            ('vect', vectorizer),
            ('actualClassifier', classifier )
        ])
        text_clf.fit(X_train, y_train)
        docs_test = X_test
        predicted = text_clf.predict(docs_test)
        print(classifier_names[i],np.mean(predicted == y_test))

In [64]:
# default BOW
runClassifiers(CountVectorizer(analyzer='word',binary=True),X_train,y_train,X_test)

In [65]:
runClassifiers(CountVectorizer(analyzer='word',binary=True,ngram_range=(1,2)),X_train,y_train,X_test)

In [ ]:
runClassifiers(CountVectorizer(analyzer='word',binary=True,ngram_range=(2,2)),X_train,y_train,X_test)

In [ ]:
# default TFIDF
runClassifiers(TfidfVectorizer(),X_train,y_train,X_test)

In [ ]:
runClassifiers(TfidfVectorizer(use_idf=True,smooth_idf=True,ngram_range=(1,1),stop_words='english'),X_train,y_train,X_test)

## Multibinomial

In [67]:
model_nb = Pipeline([
    ('vect', TfidfVectorizer(use_idf=True,smooth_idf=True,ngram_range=(1,2),stop_words='english')),
    ('multinomialNB', MultinomialNB(alpha=1))
])

In [68]:
save_prediction_accuracy(model_nb,X_train,y_train,X_test,y_test)

MultinomialNB(alpha=1, class_prior=None, fit_prior=True)

## Logistic Regression

In [71]:
#model_lr = LogisticRegression()
model_lr = Pipeline([
    ('vect', CountVectorizer(analyzer='word',binary=True)),
    ('logisticReg', LogisticRegression())
])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [72]:
save_prediction_accuracy(model_lr,X_train,y_train,X_test,y_test)

Model Performance
Accuracy = 96.00%.


In [ ]:
## KNN

In [ ]:
knn = KNeighborsClassifier(n_neighbors=3, weights = 'distance',p = 1)

In [ ]:
## Random Forest

In [ ]:
## SGD

In [ ]:
#sgd = SGDClassifier(loss='hinge', penalty='l2', alpha=0.001, random_state=42, max_iter=5, tol=None)
#ESTE CREO QUE CORRIA MEJOR CON EL TEXTO LEMMATIZED [probar, je]
model_sgd = Pipeline([
    ('vect', CountVectorizer(analyzer='word',binary=True,ngram_range=(1,2))),
    ('sgd', SGDClassifier(loss='hinge', penalty='l2', alpha=0.001, random_state=42, max_iter=5, tol=None))
])

In [ ]:
save_prediction_accuracy(model_sgd,X_train,y_train,X_test,y_test)

In [ ]:
# Ensambles

In [ ]:
## Stacking

In [ ]:
sclf = StackingClassifier(classifiers=[model_nb,model_lr],meta_classifier=SGDClassifier(loss='hinge', penalty='l2', alpha=0.001, random_state=42, max_iter=5, tol=None)) #,model_mlp

In [ ]:
save_prediction_accuracy(sclf,X_train,y_train,X_test,y_test)

In [ ]:
## Boosting

In [ ]:
### AdaBoost

In [ ]:
save_prediction_accuracy(sclf,X_train,y_train,X_test,y_test)

In [ ]:
save_prediction_accuracy(abc,X_train,y_train,X_test,y_test)

In [ ]:
## Voting Classifier

In [ ]:
evc = VotingClassifier(estimators= [('lr',model_lr),('nb',model_nb),('sgd',model_sgd)], voting = 'hard')

In [ ]:
save_prediction_accuracy(evc,X_train,y_train,X_test,y_test)

# PREDICT ON TEST AND SAVE

In [ ]:
evc.fit(train.text,y)

In [ ]:
save_prediction(sclf, test.text, 'Voting_text_not_lemmatized_without_tagsnlinks20200806_0900')

# Vectorizacion del texto con Word2vec

In [82]:
import gensim
url = 'GoogleNews-vectors-negative300.bin.gz'
embeddings = gensim.models.KeyedVectors.load_word2vec_format(url,binary=True)
data = pd.DataFrame()
for frase in train.text:
    temp = pd.DataFrame()
    for word in frase.split(' '):
        try:
            word_vec = embeddings[word]
            temp = temp.append(pd.Series(word_vec), ignore_index=True)
        except:
            pass
    mean = temp.mean()
    data = data.append(mean, ignore_index=True)
data.shape

(7613, 300)

In [83]:
pd.isnull(data).sum().sum()

5700

In [84]:
data['target'] = train['target']
data = data.dropna()

In [86]:
train_X, test_X, train_y, test_y = train_test_split(data.drop('target',axis=1), data['target'], test_size=0.2, random_state=1)
train_X.shape, test_X.shape, train_y.shape, test_y.shape

((6075, 300), (1519, 300), (6075,), (1519,))

# Modelos

In [ ]:
from sklearn.neural_network import MLPClassifier

In [ ]:
mlp = MLPClassifier()

In [ ]:
save_prediction_accuracy 

In [1]:
from sklearn.ensemble import AdaBoostClassifier

model=AdaBoostClassifier(n_estimators=800, random_state=1)
model.fit(train_X,train_y)

predictions = model.predict(test_X)
accuracy = round(accuracy_score(test_y,predictions)*100)
print('Model Performance')
print('Accuracy = {:0.2f}%.'.format(accuracy))

KeyboardInterrupt: 

In [92]:
model = LogisticRegression()
model.fit(train_X,train_y)

predictions = model.predict(test_X)
accuracy = round(accuracy_score(test_y,predictions)*100)
print('Model Performance')
print('Accuracy = {:0.2f}%.'.format(accuracy))

Model Performance
Accuracy = 78.00%.


In [93]:
model = RandomForestClassifier()
model.fit(train_X,train_y)

predictions = model.predict(test_X)
accuracy = round(accuracy_score(test_y,predictions)*100)
print('Model Performance')
print('Accuracy = {:0.2f}%.'.format(accuracy))

Model Performance
Accuracy = 78.00%.
